In [110]:
from collections import defaultdict
import pandas as pd
import numpy as np
import sqlite3
import pdb
import os

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.templates
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import seaborn as sns

import sqlite3, os
from tkinter import ON
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

Inpatient Descriptive Statistics 

In [158]:
inpatient_df = pd.read_csv('idea-1/medicare-data/medicare-data/Inpatient_provdr.csv')

In [97]:
inpatient_df.describe()

,Provider Id,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
count,163065.000000,163065.000000,163065.000000,163065.000000,163065.000000,163065.000000
mean,255569.865428,47938.121908,42.776304,36133.958434,9707.477867,8494.495174
std,151563.671767,27854.323080,51.104042,35065.365905,7664.642652,7309.467247
min,10001.000000,1040.000000,11.000000,2459.400000,2673.000000,1148.909091
25%,110092.000000,27261.000000,17.000000,15947.166670,5234.500000,4192.354839
50%,250007.000000,44309.000000,27.000000,25245.823530,7214.100000,6158.465116
75%,380075.000000,72901.000000,49.000000,43232.592590,11286.400000,10056.882350
max,670077.000000,99835.000000,3383.000000,929118.909100,156158.181800,154620.818200


In [98]:
inpatient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   DRG Definition                              163065 non-null  object 
 1   Provider Id                                 163065 non-null  int64  
 2   Provider Name                               163065 non-null  object 
 3   Provider Street Address                     163065 non-null  object 
 4   Provider City                               163065 non-null  object 
 5   Provider State                              163065 non-null  object 
 6   Provider Zip Code                           163065 non-null  int64  
 7   Hospital Referral Region (HRR) Description  163065 non-null  object 
 8   Total Discharges                            163065 non-null  int64  
 9   Average Covered Charges                     163065 non-null  float64
 

In [99]:
inpatient_df.corr()

,Provider Id,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
Provider Id,1.000000,-0.160237,0.000456,-0.112490,-0.040059,-0.044846
Provider Zip Code,-0.160237,1.000000,-0.071804,0.140658,0.027496,0.022262
Total Discharges,0.000456,-0.071804,1.000000,-0.020766,-0.016020,-0.021299
Average Covered Charges,-0.112490,0.140658,-0.020766,1.000000,0.774112,0.768927
Average Total Payments,-0.040059,0.027496,-0.016020,0.774112,1.000000,0.989362
Average Medicare Payments,-0.044846,0.022262,-0.021299,0.768927,0.989362,1.000000


Inpatient Top 10 Analysis

In [218]:
#Inpatient Provider City
city_count  = inpatient_df['Provider City'].value_counts()
city_count = city_count[:10,]
fig = go.Figure()
fig = px.bar(x = city_count.index, y = city_count.values)
fig.update_layout(
    title='Inpatient Provider in Top 10 cities',
    template='ggplot2',
    xaxis=dict(
        title='Provider City Id',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
)
fig.layout.xaxis.type = 'category'
fig.show()


In [101]:
# Inpatient by State
state_count  = inpatient_df['Provider State'].value_counts()
state_count = state_count[:10,]
fig = px.histogram(x = state_count.index, y = state_count.values)
fig.update_layout(
    title='Inpatient Provider in Top 10 States',
    template='ggplot2',
    xaxis=dict(
        title='States',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig.layout.xaxis.type = 'category'
fig.show()


In [102]:
# Inpatient by Zip Code
zipcode_count  = inpatient_df['Provider Zip Code'].value_counts()
zipcode_count = zipcode_count[:10,]
fig = px.histogram(x = zipcode_count.index, y = zipcode_count.values)
fig.update_layout(
    title='Inpatient Provider in Top 10 Zip Codes',
    template='ggplot2',
    xaxis=dict(
        title='Zip Codes',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig.layout.xaxis.type = 'category'
fig.show()


Inpatient procedures COST among states

In [160]:
class SQLServer:
      def __init__(self, server):
        self.server = server
       
      def __enter__(self):
        self.conn: sqlite3.Connection = sqlite3.connect(self.server)
        self.cur: sqlite3.Cursor = self.conn.cursor() 
        return self
        
      def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.close()

In [195]:
_df = pd.read_csv('idea-1/medicare-data/medicare-data/Inpatient_provdr.csv')
inpatient_hospital_charges = 'inpatient_hospital_charges.db'

with SQLServer('inpatient_hospital_charges') as db:
    _df.columns = [column.strip() for column in _df.columns]
    _df.to_sql('inpatient_hospital_charges', db.conn, if_exists='replace', index=False)

In [ ]:
with SQLServer('inpatient_hospital_charges') as db:
    query = '''
SELECT `DRG Definition`, `Provider State` as providerState, t.maxAvgPaymentPerDRG
FROM inpatient_hospital_charges hc
INNER JOIN (
    SELECT `DRG Definition` as drg, MAX(`Average Total Payments`) as maxAvgPaymentPerDRG
    FROM inpatient_hospital_charges
    GROUP BY `DRG Definition`
) t
ON hc.`DRG Definition` == t.drg AND hc.`Average Total Payments` == t.maxAvgPaymentPerDRG
ORDER BY providerState ASC
'''
    cursor = db.cur.execute(query)
    results = [record for record in cursor]

In [ ]:
_df = pd.DataFrame(results, columns=['DRG', 'ProviderState', 'Payment'])

In [ ]:
with SQLServer('inpatient_hospital_charges') as db:
    ranking_by_drg = dict()
    for drg in inpatient_df['DRG Definition'].unique():

        query = """
            SELECT `Provider State` as providerState
            FROM inpatient_hospital_charges
            WHERE `DRG Definition` = '{drg}' 
            GROUP BY `Provider State`
            ORDER BY AVG(`Average Total Payments`) ASC
        """.format(drg=drg)
    
        cursor = db.cur.execute(query)
        ranking_by_drg[drg] = [record[0] for record in cursor]


In [204]:
for k,v in ranking_by_drg.items():
    while True:
        if len(v) >= 51:
            break
        v.append(None)

In [205]:
drg_by_id = {key.split(' - ')[0]: key for key in ranking_by_drg.keys()}
df_rank = pd.DataFrame(ranking_by_drg)

In [ ]:
_df = pd.DataFrame(index=inpatient_df['Provider State'].unique())
for column in df_rank.columns:
    rankings_by_state = []

    if column in ['rank']:
        continue
        
    for rank, curr_state in zip(inpatient_df.index, df_rank[column]):
        rankings_by_state.append(curr_state)
    
    t = pd.DataFrame(rankings_by_state, columns=['State'])
    t['Rank'] = t.index
    t.set_index(['State'], inplace=True)
    
    _df = pd.merge(left=_df, right=t, how='left', left_index=True, right_index=True)
    
_df.columns = drg_by_id.keys()

In [207]:
_df.fillna(-100, inplace=True)
_df.head()

,039,057,064,065,066,069,074,101,149,176,...,379,389,390,391,392,394,418,419,439,460
AK,45,46,50,50,50,49.0,-100.0,49,-100.0,49,...,50,50,-100.0,-100.0,50,44,49.0,47,-100.0,50
AL,0,0,1,1,0,0.0,2.0,0,2.0,0,...,2,0,1.0,0.0,1,0,0.0,1,0.0,0
AR,3,4,3,2,2,1.0,5.0,3,1.0,1,...,4,4,2.0,3.0,2,2,2.0,2,1.0,1
AZ,36,29,33,33,34,36.0,22.0,34,27.0,35,...,35,35,37.0,25.0,35,30,38.0,30,19.0,30
CA,44,44,47,45,46,44.0,44.0,44,40.0,45,...,44,46,46.0,46.0,45,48,47.0,45,43.0,46


In [217]:
fig = px.imshow(_df,text_auto=True, aspect="auto")
fig.update_layout(
    title='Inpatient Cost Rankings Per Procedure',
    template='ggplot2',
    xaxis=dict(
        title='DRG Definition Id',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='State',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1 
)
fig.show()